In [3]:
import pandas as pd

# Membaca Dataset
data = pd.read_csv('insurance.csv')

# Menampilkan dataset
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# **Preprocessing Data**

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


In [6]:
data.shape

(1338, 7)

In [7]:
data.isnull().sum()

,0
age,0
sex,0
bmi,0
children,0
smoker,0
region,0
charges,0


# **Split Data**

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Bagi kolom variabel independen dan dependen
X = data.drop(columns=['charges'])
y = data['charges']

# ubah kolom kategorik menjadi numerik
X_encoded = pd.get_dummies(X, columns=['sex', 'smoker', 'region'], drop_first=True)

# splitting data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# **Regresi**

In [9]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# Model linear regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# nilai prediksi
y_pred = linear_model.predict(X_test)

# Hasil dari regresi
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 33596915.85136145
R-squared: 0.7835929767120724


Mean Squared Error (MSE)= 33,596,915.85 menunjukkan rata-rata kesalahan kuadrat antara nilai aktual dan prediksi model. Nilai ini besar karena target (charges) memiliki skala besar (hingga puluhan ribu).

R-squared (R²) = 0.784 menunjukkan bahwa model Anda mampu menjelaskan 78.4% variabilitas dari data target (charges) berdasarkan fitur yang digunakan.
Semakin mendekati 1, semakin baik model merepresentasikan hubungan antara variabel input (X) dan target (y).


# **Classification**

In [10]:
# ubah kolom 'charges' menjadi kategori (tinggi/rendah)
data['charges_category'] = pd.cut(data['charges'], bins=2, labels=['rendah', 'tinggi']) # Adjust bins as needed

# Define features (X) and target variable (y) for classification
X_class = data.drop(columns=['charges', 'charges_category'])
y_class = data['charges_category']

# ubah kolom kategorik menjadi numerik
X_class_encoded = pd.get_dummies(X_class, columns=['sex', 'smoker', 'region'], drop_first=True)

# Split data menjadi train dan test
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class_encoded, y_class, test_size=0.2, random_state=42
)

# **Decision Tree**

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

# Model decision tree
model_dt = DecisionTreeClassifier(random_state=42)

# Hasil
dt = cross_val_score(model_dt, X_train_class, y_train_class, cv=5)
print("Decision Tree Accuracy:", dt)
print("Mean cross validation score : ",dt.mean())
print("Standard deviation of cross validation score :",dt.std())

Decision Tree Accuracy: [0.99065421 0.97196262 1.         0.98130841 0.97663551]
Mean cross validation score :  0.9841121495327103
Standard deviation of cross validation score : 0.010065728611466356


Decision Tree Accuracy:
Akurasi model Decision Tree pada tiap fold dari cross-validation adalah

- Fold 1: 0.99065421
- Fold 2: 0.97196262
- Fold 3: 1.0
- Fold 4: 0.98130841
- Fold 5: 0.97663551
Nilai ini menunjukkan tingkat akurasi model pada masing-masing fold saat melakukan prediksi terhadap data validasi.

Mean Cross-Validation Score:
Rata-rata akurasi dari kelima fold adalah 0.9841121495327103 atau sekitar 98.41% menunjukkan performa rata-rata model dalam memprediksi data validasi selama proses cross-validation. Nilai yang tinggi ini mengindikasikan model memiliki performa yang sangat baik.

Standard Deviation of cross validation Score:
Standar deviasi akurasi pada kelima fold adalah 0.010065728611466356.
Standar deviasi yang rendah ini menunjukkan bahwa performa model cukup konsisten di setiap fold, sehingga model dapat dianggap stabil.

# **Naive Bayes**

In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# inisialisasi naive bayes classifier
gnb = GaussianNB()
gnb.fit(X_train_class, y_train_class)

# buat naive bayes prediksi
y_pred_gnb = gnb.predict(X_test_class)

# Evaluasi model
nb = cross_val_score(gnb, X_train_class, y_train_class, cv=5)
print("Naive Bayes Accuracy:", nb)
print("Mean accuracy score : ",nb.mean())
print("Standard deviation of accuracy score :",nb.std())


Naive Bayes Accuracy: [0.95794393 0.91121495 0.92990654 0.91588785 0.94859813]
Mean accuracy score :  0.9327102803738316
Standard deviation of accuracy score : 0.018122167691276007


Akurasi model pada masing-masing dari 5 fold adalah:
- Fold 1: 0.95794393
- Fold 2: 0.91121495
- Fold 3: 0.92990654
- Fold 4: 0.91588785
- Fold 5: 0.94859813

Akurasi ini mencerminkan performa model pada data validasi untuk setiap fold. Nilai-nilai tersebut menunjukkan hasil yang cukup konsisten di seluruh fold.

Mean Accuracy Score
Rata-rata akurasi dari 5 fold adalah 0.9327120823783816 atau sekitar 93.27%.
Hal ini mengindikasikan bahwa model Naive Bayes memiliki performa prediksi yang cukup baik secara keseluruhan selama proses cross-validation.

Standard Deviation of Accuracy Score
Standar deviasi akurasi adalah 0.018122167691276007 atau sekitar 1.81%.
Standar deviasi yang rendah ini menunjukkan bahwa performa model cukup konsisten di seluruh fold, tanpa adanya fluktuasi yang besar dalam hasil akurasi.

# **SVM(SUPPORT VECTOR MACHINES)**

In [14]:
from sklearn.svm import SVC

# Model SVM
svm_model = SVC(random_state=42)

# Hasil
svm = cross_val_score(svm_model, X_train_class, y_train_class, cv=5)
print("SVM Accuracy:", svm)
print("Mean cross validation score : ",svm.mean())
print("Standard deviation of cross validation score :",svm.std())

SVM Accuracy: [0.88785047 0.88785047 0.88785047 0.88317757 0.88317757]
Mean cross validation score :  0.8859813084112149
Standard deviation of cross validation score : 0.0022892427502646255




Hasil akurasi dari SVM cross-validation pada 5 fold adalah
  - 0.88785047
  - 0.88785047
  - 0.88785047
  - 0.88317757
  - 0.88317757

Ini menunjukkan bahwa model SVM yang Anda bangun memiliki akurasi yang konsisten di sekitar 88.78% hingga 88.32% pada setiap fold dari cross-validation.

Mean Cross Validation Score:
Rata-rata akurasi dari cross-validation adalah 0.8859813084112149.
Ini berarti bahwa secara rata-rata, model SVM Anda memiliki akurasi sekitar 88.60% ketika diuji pada data yang berbeda selama proses cross-validation.
Standard Deviation of Cross Validation Score:

Standar deviasi dari akurasi cross-validation adalah 0.0022892427502646255.
Standar deviasi yang rendah ini menunjukkan bahwa variasi akurasi antara fold yang berbeda sangat kecil, yang berarti model menghasilkan performa yang cukup konsisten di berbagai subset data.

# **Random Forest**

In [15]:
from sklearn.ensemble import RandomForestClassifier

# Model Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Hasil
rf = cross_val_score(rf_model, X_train_class, y_train_class, cv=5)
print("Random Forest Accuracy:", rf)
print("Mean cross validation score : ", rf.mean())
print("Standard deviation of cross validation score :", rf.std())

Random Forest Accuracy: [0.9953271  0.98598131 1.         0.98598131 0.98598131]
Mean cross validation score :  0.9906542056074766
Standard deviation of cross validation score : 0.005910799364800692


Random Forest Accuracy:

Hasil akurasi dari cross-validation pada 5 fold adalah
- 0.9953271
- 0.98598131
- 1
- 0.98598131
- 0.98598131

Ini menunjukkan bahwa model Random Forest memiliki akurasi yang sangat tinggi, berkisar antara 98.60% hingga 100% pada setiap fold dari cross-validation.

Mean Cross Validation Score:
Rata-rata akurasi dari cross-validation adalah 0.9906542056074766.
Ini berarti bahwa secara rata-rata, model Random Forest Anda memiliki akurasi sekitar 99.07% ketika diuji pada data yang berbeda selama proses cross-validation.

Standard Deviation of Cross Validation Score:
Standar deviasi dari akurasi cross-validation adalah 0.005910799364800692.
Standar deviasi yang rendah ini menunjukkan bahwa variasi akurasi antara fold yang berbeda sangat kecil, yang berarti model Anda menghasilkan performa yang sangat konsisten di berbagai subset data.

#**KNN(K-Nearest Neighbors)**

In [22]:
from sklearn.neighbors import KNeighborsClassifier

# Model KNN
knn_model = KNeighborsClassifier(n_neighbors=5)

# Hasil
knn = cross_val_score(knn_model, X_train_class, y_train_class, cv=5)
print("KNN Accuracy:", knn)
print("Mean cross validation score : ", knn.mean())
print("Standard deviation of cross validation score :", knn.std())

KNN Accuracy: [0.89252336 0.89252336 0.86915888 0.89252336 0.86448598]
Mean cross validation score :  0.8822429906542057
Standard deviation of cross validation score : 0.012677252304907058


KNN Accuracy:

Hasil akurasi dari cross-validation pada 5 fold adalah
- 0.89252336
- 0.89252336
- 0.86915888
- 0.89252336
- 0.86448598

Ini menunjukkan bahwa model KNN memiliki akurasi yang bervariasi antara 86.45% hingga 89.25% pada setiap fold dari cross-validation.

Mean Cross Validation Score:
Rata-rata akurasi dari cross-validation adalah 0.8822429906542057.
Ini berarti bahwa secara rata-rata, model KNN Anda memiliki akurasi sekitar 88.22% ketika diuji pada data yang berbeda selama proses cross-validation.

Standard Deviation of Cross Validation Score:
Standar deviasi dari akurasi cross-validation adalah 0.012677252304907058.
Standar deviasi ini menunjukkan bahwa variasi akurasi antara fold yang berbeda sekitar 1.27%, yang menunjukkan bahwa model menghasilkan performa yang cukup konsisten di berbagai subset data, meskipun ada sedikit variasi.

## **Kesimpulan**
dengan hasil dari kelima algoritma yang digunakan algoritma **Random Forest** adalah algoritma dengan performa terbaik di antara kelima model, dengan akurasi yang sangat tinggi dan konsisten lalu, Decision Tree juga menunjukkan performa yang sangat baik dan konsisten, sedikit di bawah Random Forest. Naive Bayes dan SVM menunjukkan performa yang baik dengan akurasi yang cukup tinggi dan konsisten. algoritma terakhir yaitu KNN memiliki performa yang baik, meskipun sedikit lebih rendah dibandingkan dengan model lainnya, tetapi masih dapat diandalkan.